# 章節 15：維度處理

## 15.6 折疊階層式結構

所有下層的分佈都是用相同的資料更新，所以前 m 個參數都是相同的。我們可以移除下層的 Dirichlet 分佈假設並且合併到上層。Species2 實作這個優化：

<pre>
class Species2(object):
    def __init__(self, ns):
        # ns 是多少物種的陣列
        self.ns = ns
        
        # 一序列對應的機率
        self.probs = numpy.ones(len(ns), dtype=numpy.double)
        
        # Dirichlet 的參數，初始化都為 1
        self.params = numpy.ones(self.high, dtype=numpy.int)
        
</pre>

Species2.Update 更新所有階層：先更新每個物種 n 的機率，再來更新 Dirichlet 參數：

<pre>
# class Species2
    def Update(self, data):
        like = numpy.zeros(len(self.ns), dtype=numpy.double)
        for i in range(1000):
            like += self.SampleLikelihood(data)
        self.probs *= like
        self.probs /= self.probs.sum()
        
        # 更新 Dirichlet 參數
        m = len(data)
        self.params[:m] += data
</pre>

- SampleLikelihood 方法回傳對每個 n 值似然性的陣列。並且將 1000 次取樣的似然性結果加總。
- self.probs 乘上加總的似然性並且正規化。
- 最後兩行是更新參數，就是 Dirichlet.Update 方法中做的事情



Now let’s look at SampleLikelihood. There are two opportunities for opti- mization here:
• When the hypothetical number of species, n, exceeds the observed number, m, we only need the first m terms of the multinomial PMF; the rest are 1.
• If the number of species is large, the likelihood of the data might be too small for floating-point (see 10.5). So it is safer to compute log- likelihoods.
Again, the multinomial PMF is
c px1 ···pxn
x1n
So the log-likelihood is
log cx + x1 log p1 + · · · + xn log pn
which is fast and easy to compute. Again, cx it is the same for all hypotheses, so we can drop it. Here’s the code:
# class Species2
    def SampleLikelihood(self, data):
        gammas = numpy.random.gamma(self.params)
        m = len(data)
        row = gammas[:m]
        col = numpy.cumsum(gammas)
        log_likes = []
        for n in self.ns:
            ps = row / col[n-1]
            terms = data * numpy.log(ps)
            log_like = terms.sum()
            log_likes.append(log_like)
        log_likes -= numpy.max(log_likes)
        likes = numpy.exp(log_likes)
        coefs = [thinkbayes.BinomialCoef(n, m) for n in self.ns]
        likes *= coefs
        
        
return likes
gammas is an array of values from a gamma distribution; its length is the largest hypothetical value of n. row is just the first m elements of gammas; since these are the only elements that depend on the data, they are the only ones we need.
For each value of n we need to divide row by the total of the first n values from gamma. cumsum computes these cumulative sums and stores them in col.
The loop iterates through the values of n and accumulates a list of log- likelihoods.
Inside the loop, ps contains the row of probabilities, normalized with the appropriate cumulative sum. terms contains the terms of the summation, xi log pi, and log_like contains their sum.
After the loop, we want to convert the log-likelihoods to linear likelihoods, but first it’s a good idea to shift them so the largest log-likelihood is 0; that way the linear likelihoods are not too small (see 10.5).
Finally, before we return the likelihood, we have to apply a correction factor, which is the number of ways we could have observed these m species, if the total number of species is n. BinomialCoefficient computes “n choose m”, which is written (mn ).
As often happens, the optimized version is less readable and more error- prone than the original. But that’s one reason I think it is a good idea to start with the simple version; we can use it for regression testing. I plotted results from both versions and confirmed that they are approximately equal, and that they converge as the number of iterations increases.